# 0. Functions

## 0.1 Request Data From API

In [ ]:
#############################################################
## Author: Hunter Hopkins
##
## Description: Macro function to request data from the
##              last.fm API. Automatically fills in API
##              key, format, and user agent fields. 
##
## Argument(s):
##  params: list - contains extra arguments for API GET request
##  userAgent: string - name of user agent to pass to GET request
##
## Returns: response
#############################################################
get.lastfm <- function(params, userAgent) {
  require(httr)
  params$api_key <- APIKey
  params$format <- "json"
  
  res <- GET(url, user_agent(userAgent), query = params)
  return(res)
}

## 0.2 Convert Image Link to HTML Tag

In [ ]:
#############################################################
## Author: Hunter Hopkins
##
## Description: Converts image URL link to be used with html img tag. 
##              Used for displaying table images.
##
## Argument(s):
##  imgLink: vector - contains image URL links
##
## Returns: string
#############################################################
img.link.to.html <- function(imgLink) {
  vapply(imgLink, function(x) paste0('<img src="', x, '" height="52"></img>'), character(1))
}

## 0.3 Filter Data to Given Time Period

In [ ]:
#############################################################
## Author: Hunter Hopkins
##
## Description: Filters data.frame to a specific time period. 
##              Used to get data for each tab in the dashboard.
##
## Argument(s):
##  df: data.frame - data.frame containing scrobbles
##  timePeriod: string - time period you want to filter to
##                       "week", "month", or "year"
##
## Returns: data.frame
#############################################################
filter.to.time.period <- function(df, timePeriod) {
  require(dplyr) # uses filter() function
  require(lubridate) # ez date subtraction
  
  if (tolower(timePeriod) != "week" & 
      tolower(timePeriod) != "month" & 
      tolower(timePeriod) != "year") {
    stop("timePeriod must be 'week', 'month', or 'year'")
  }
  
  switch(tolower(timePeriod),
         week = {
           # These values are duplicated in a vector because the same week can have two values (see below if statements)
           # Because of this I need to check both possible indices.
           # If these aren't duplicated, the second value is NA, and the ifelse statement below doesn't work
           lastWeek <- c(format(Sys.time() - weeks(1), "%U"), format(Sys.time() - weeks(1), "%U"))
           twoWeeks <- c(format(Sys.time() - weeks(2), "%U"), format(Sys.time() - weeks(2), "%U"))
           # Possible to have a week 00 if week 52 continues into the new year
           # These are the same week 
           if (any(lastWeek == "00" | lastWeek == "52")) {
             lastWeek == c("00", "52")
           }
           if (any(twoWeeks == "00" | twoWeeks == "52")) {
             twoWeeks == c("00", "52")
           }
           
           weekCompScrobbles <- df %>% filter(format(date, "%U") == lastWeek[1] |
                                                format(date, "%U") == lastWeek[2] |
                                                format(date, "%U") == twoWeeks[1] |
                                                format(date, "%U") == twoWeeks[2])
           
           weekCompScrobbles$week <- ifelse(format(weekCompScrobbles$date, "%U") == lastWeek[1] | 
                                              format(weekCompScrobbles$date, "%U") == lastWeek[2],
                                            0,
                                            1)
           weekCompScrobbles$week <- factor(weekCompScrobbles$week)
           return(weekCompScrobbles)
         },
         month = {
           lastMonth <- format(Sys.time() %m-% months(1), "%m")
           twoMonths <- format(Sys.time() %m-% months(2), "%m")
           
           monthCompScrobbles <- df %>% filter(format(date, "%m") == lastMonth |
                                                 format(date, "%m") == twoMonths)
           monthCompScrobbles$month <- ifelse(format(monthCompScrobbles$date, "%m") == lastMonth,
                                              0,
                                              1)
           monthCompScrobbles$month <- factor(monthCompScrobbles$month)
           return(monthCompScrobbles)
         },
         year = {
           lastYear <- format(Sys.time() - years(1), "%Y")
           twoYears <- format(Sys.time() - years(2), "%Y")

           yearCompScrobbles <- df %>% filter(format(date, "%Y") == lastYear |
                                                format(date, "%Y") == twoYears)
           yearCompScrobbles$year <- ifelse(format(yearCompScrobbles$date, "%Y") == lastYear,
                                            0,
                                            1)
           yearCompScrobbles$year <- factor(yearCompScrobbles$year)
           return(yearCompScrobbles)
         })
}

## 0.4 Get Top Artists for a Given Time Period

In [ ]:
#############################################################
## Author: Hunter Hopkins
##
## Description: Gets n most frequent to artists for a specific
##              time period. Used to create tables.
##
## Argument(s):
##  periodDF: data.frame - data.frame containing scrobbles
##  period: string - time period you want to filter to
##                   "week", "month", or "year"
##  n: numeric - number of artists to return
##
## Returns: data.frame
#############################################################
get.period.top.artists <- function(periodDF, period, n) {
  require(dplyr) # uses filter() function
  
  # error if wrong time period requested
  if (tolower(period) != "week" & 
      tolower(period) != "month" & 
      tolower(period) != "year") {
    stop("period must be 'week', 'month', or 'year'")
  }
  
  # filter df based on requested time period
  switch(tolower(period),
         week = thisPeriod <- periodDF %>% filter(week == 0),
         month = thisPeriod <- periodDF %>% filter(month == 0),
         year = thisPeriod <- periodDF %>% filter(year == 0))
  
  
  # get top artists
  topArtists <- head(count(thisPeriod, artistName, sort = T), n)
  topArtists$img <- character(n)
  topArtists <- topArtists[, c(3, 1, 2)]
  
  # Get top tracks. API doesn't return artist/album images so most listened to track image is used in place
  topTracks <- vapply(topArtists$artistName, 
                      function(x) count(subset(thisPeriod, artistName == x), trackName, sort = T)[1, 1], 
                      list(1))
  
  # find image for top tracks
  for (i in 1:n) {
    topArtists$img[i] <- periodDF$trackImage[which(periodDF$artistName == topArtists$artistName[i] & 
                                                     periodDF$trackName == topTracks[i])][1]
  }
  
  # convert image links to html tag
  topArtists$img <- img.link.to.html(topArtists$img)
  
  names(topArtists) <- c("Image", "Artist", "Plays")
  return(topArtists)
}

## 0.5 Get Top Albums for a Given Time Period

In [ ]:
#############################################################
## Author: Hunter Hopkins
##
## Description: Gets n most frequent to albums for a specific
##              time period. Used to create tables.
##
## Argument(s):
##  periodDF: data.frame - data.frame containing scrobbles
##  period: string - time period you want to filter to
##                   "week", "month", or "year"
##  n: numeric - number of albums to return
##
## Returns: data.frame
#############################################################
get.period.top.albums <- function(periodDF, period, n) {
  require(dplyr) # uses filter() function
  
  # error if wrong time period requested
  if (tolower(period) != "week" & 
      tolower(period) != "month" & 
      tolower(period) != "year") {
    stop("period must be 'week', 'month', or 'year'")
  }
  
  # filter df based on requested time period
  switch(tolower(period),
         week = thisPeriod <- periodDF %>% filter(week == 0),
         month = thisPeriod <- periodDF %>% filter(month == 0),
         year = thisPeriod <- periodDF %>% filter(year == 0))
  
  # get top albums
  topAlbums <- head(count(thisPeriod, albumName, artistName, sort = T), n)
  topAlbums$img <- character(n)
  topAlbums <- topAlbums[, c(4, 2, 1, 3)]
  
  # Get top tracks. API doesn't return artist/album images so most listened to track image is used in place
  topTracks <- vapply(topAlbums$albumName, 
                      function(x) count(subset(thisPeriod, albumName == x), trackName, sort = T)[1, 1], 
                      list(1))
  # find image for top tracks
  for (i in 1:n) {
    topAlbums$img[i] <- periodDF$trackImage[which(periodDF$albumName == topAlbums$albumName[i] & 
                                                     periodDF$trackName == topTracks[i])][1]
  }
  
  # convert image links to html tag
  topAlbums$img <- img.link.to.html(topAlbums$img)
  
  names(topAlbums) <- c("Image", "Artist", "Album", "Plays")
  return(topAlbums)
}

## 0.6 Get Top Tracks for a Given Time Period

In [ ]:
#############################################################
## Author: Hunter Hopkins
##
## Description: Gets n most frequent to tracks for a specific
##              time period. Used to create tables.
##
## Argument(s):
##  periodDF: data.frame - data.frame containing scrobbles
##  period: string - time period you want to filter to
##                   "week", "month", or "year"
##  n: numeric - number of tracks to return
##
## Returns: data.frame
#############################################################
get.period.top.tracks <- function(periodDF, period, n) {
  require(dplyr) # uses filter() function
  
  # error if wrong time period requested
  if (tolower(period) != "week" & 
      tolower(period) != "month" & 
      tolower(period) != "year") {
    stop("period must be 'week', 'month', or 'year'")
  }
  
  # filter df based on requested time period
  switch(tolower(period),
         week = thisPeriod <- periodDF %>% filter(week == 0),
         month = thisPeriod <- periodDF %>% filter(month == 0),
         year = thisPeriod <- periodDF %>% filter(year == 0))
  
  # Get top tracks
  topTracks <- head(count(thisPeriod, trackName, artistName, sort = T), n)
  topTracks$img <- character(n)
  topTracks <- topTracks[, c(4, 2, 1, 3)]
  
  # get track images
  for (i in 1:n) {
    topTracks$img[i] <- periodDF$trackImage[which(periodDF$artistName == topTracks$artistName[i] & 
                                                    periodDF$trackName == topTracks$trackName[i])][1]
  }
  
  # convert image link to html tag
  topTracks$img <- img.link.to.html(topTracks$img)

  names(topTracks) <- c("Image", "Artist", "Track", "Plays")
  return(topTracks)
}

## 0.7 Convert Decimal Week to Range of Dates
e.g. week 1 of 2022 becomes "Jan 02-Jan 08"

In [ ]:
#############################################################
## Author: Hunter Hopkins
##
## Description: Converts decimal week to range of dates.
##              Used in weekly chart legend.
##
## Argument(s):
##  periodDF: data.frame - data.frame containing scrobbles
##  weekNum: specifies if you want last week (0) or two
##           weeks ago (1)
##
## Returns: character vector
#############################################################
week.to.date.range <- function(periodDF, weekNum) {
  require(dplyr) # uses filter() function
  
  dates <- periodDF %>% filter(week == weekNum)
  dates <- dates$date
  dateRange <- c(dates[length(dates)], dates[1])
  dateRangeStr <- paste0(format(dateRange[1], "%b %d"), "-", format(dateRange[2], "%b %d"))
  return(dateRangeStr)
}